In [98]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import folium
from folium.plugins import TimestampedGeoJson, HeatMapWithTime

pd.options.display.max_colwidth = 200

In [99]:
file_name = 'results/greedy_50_vehicles_50_bookings_linear.csv.gz'

data = pd.read_csv(file_name, sep=';',
                   # convert column to dict
                   converters={'details': lambda v: eval(v)})

print(f'Data shape {data.shape}')

# Use only 10% of the whole dataset
# data = data[:int(data.shape[0] * 0.1)]


data.head()

Data shape (59500, 9)


,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,0,NaN,offline,idling,-73.99716,40.71196,{}
1,0,vehicle,1,NaN,offline,idling,-73.98232,40.76953,{}
2,0,vehicle,2,NaN,offline,idling,-73.98243,40.78500,{}
3,0,vehicle,3,NaN,offline,idling,-74.00742,40.72601,{}
4,0,vehicle,4,NaN,offline,idling,-73.97801,40.75811,{}


In [100]:
details = data.details.apply(pd.Series)

details[~data.itinerary_id.isna()].head()

,dropoff,eta,pickup,dst,route_duration,route_distance,trip_duration,trip_distance,vid,stop
51,NaN,15.0,0.0,"{'lat': 40.73491, 'lon': -73.97696}",13.0,1.061,0.0,0.0,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN
55,NaN,10.0,1.0,"{'lat': 40.75655, 'lon': -73.99023}",17.0,1.274,0.0,0.0,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN


In [101]:
# history of one vehicle

vehicle_uuid = data[data.object_type == 'vehicle'].uuid.values[0]

vehicle_changes = data[(data.uuid == vehicle_uuid) & (data.object_type == 'vehicle')]

vehicle_changes

,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,0,NaN,offline,idling,-73.99716,40.71196,{}
5078,374,vehicle,0,e9833f2f88eb4ea293ff611484248e2f,idling,moving_to,-73.99716,40.71196,"{'eta': 17, 'pickup': 62, 'dst': {'lat': 40.71839, 'lon': -73.99066}, 'route_duration': 16, 'route_distance': 1.106, 'trip_duration': 0, 'trip_distance': 0.0}"
5359,390,vehicle,0,e9833f2f88eb4ea293ff611484248e2f,moving_to,idling,-73.99066,40.71839,"{'stop': 'arrived', 'eta': 17, 'pickup': 62, 'dst': {'lat': 40.71839, 'lon': -73.99066}, 'route_duration': 16, 'route_distance': 1.106, 'trip_duration': 16, 'trip_distance': 1.106}"
5379,390,vehicle,0,e9833f2f88eb4ea293ff611484248e2f,idling,moving_to,-73.99066,40.71839,"{'dropoff': 62, 'dst': {'lat': 40.74219, 'lon': -73.98497}, 'route_duration': 45, 'route_distance': 3.174, 'trip_duration': 0, 'trip_distance': 0.0}"
6218,435,vehicle,0,e9833f2f88eb4ea293ff611484248e2f,moving_to,idling,-73.98497,40.74219,"{'stop': 'arrived', 'dropoff': 62, 'dst': {'lat': 40.74219, 'lon': -73.98497}, 'route_duration': 45, 'route_distance': 3.174, 'trip_duration': 45, 'trip_distance': 3.174}"
...,...,...,...,...,...,...,...,...,...
55189,2057,vehicle,0,f3b85c8ff273462d99dd4e8c5f44371a,moving_to,idling,-73.96563,40.76117,"{'stop': 'change_dest', 'eta': 23, 'pickup': 284, 'dst': {'lat': 40.76069, 'lon': -73.9645}, 'route_duration': 2, 'route_distance': 0.218, 'trip_duration': 1, 'trip_distance': 0.109}"
55190,2057,vehicle,0,f3b85c8ff273462d99dd4e8c5f44371a,idling,moving_to,-73.96563,40.76117,"{'eta': 23, 'pickup': 284, 'dst': {'lat': 40.76069, 'lon': -73.9645}, 'route_duration': 1, 'route_distance': 0.109, 'trip_duration': 0, 'trip_distance': 0.0}"
55191,2058,vehicle,0,f3b85c8ff273462d99dd4e8c5f44371a,moving_to,idling,-73.96450,40.76069,"{'stop': 'arrived', 'eta': 23, 'pickup': 284, 'dst': {'lat': 40.76069, 'lon': -73.9645}, 'route_duration': 1, 'route_distance': 0.109, 'trip_duration': 1, 'trip_distance': 0.109}"
55235,2058,vehicle,0,f3b85c8ff273462d99dd4e8c5f44371a,idling,moving_to,-73.96450,40.76069,"{'dropoff': 284, 'dst': {'lat': 40.73937, 'lon': -73.98046}, 'route_duration': 36, 'route_distance': 4.903, 'trip_duration': 0, 'trip_distance': 0.0}"


In [102]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

now = datetime.now()
features = []

prev_pos = None
for item in vehicle_changes.itertuples():
    p1 = item.lon, item.lat
    
    folium.Circle(
        radius=10,
        location=[p1[1], p1[0]],
        color='green',
        fill=True,
    ).add_to(m)

#     if prev_pos:
#         p1 = prev_pos
        
#     p2 = item.lon, item.lat

#     tm = str(now + timedelta(minutes=(item.clock_time)))

#     color = 'brown'
# #     if s['details'].get('pickup'):
#     if item.details.get('pickup') is not None:
#         color = 'green'
        
        
#         folium.Circle(
#             radius=25,
#             location=[p2[-1], p2[0]],
#             color='blue',
#             fill=True,
#         ).add_to(m)
#         geom = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'LineString',
#                 'coordinates': [list(p1), list(p1)]
#             },
#             'properties': {
#                 'times': [tm, tm],
#                 'style': {
#                     'color': 'blue',
#                     'opacity': 0.6
#                 },
#                 'icon': 'circle',
#                 'iconstyle': {
#                     'radius': 25,
#                     'fillOpacity': 0.9,
#                     'iconColor': 'red'
#                 }
#             }
#         }    
# #         features.append(geom)
    
    
# #     if item.details.get('dropoff') is not None:
# #         color = 'brown'

#     geom = {
#         'type': 'Feature',
#         'geometry': {
#             'type': 'LineString',
#             'coordinates': [list(p1), list(p2)]
#         },
#         'properties': {
#             'times': [tm, tm],
#             'style': {
#                 'color': color,
#                 'opacity': 0.6
#             },
#             'icon': 'circle',
#             'iconstyle': {
#                 'radius': 7,
#                 'fillOpacity': 0.9,
#                 'iconColor': 'red'
#             }
#         }
#     }

#     features.append(geom)
    
#     prev_pos = p2

#     folium.Circle(
#         radius=100,
#         location=[p1[1], p1[0]],
#         color='green',
#         fill=False,
#     ).add_to(m)
m


In [103]:
# m = folium.Map(location=[40.76953, -73.98232])

TimestampedGeoJson(
    features,
    period='PT10M',
    add_last_point=True,
    auto_play=False, 
    loop=False, 
    max_speed=1,
    loop_button=True,
    time_slider_drag_update=True
).add_to(m)

m